In [ ]:
# %pip install googletrans == 3.1.0a0


##### Imports

In [ ]:
from googletrans import Translator
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sinling import SinhalaStemmer
import nltk
import numpy as np
import pandas as pd
import re
import string


##### Download stop words


In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords


##### Defining Sinhala stop words

In [ ]:
stopwords_sinhala = ["සහ", "සමග", "සමඟ", "අහා", "ආහ්", "ආ", "ඕහෝ", "අනේ", "අඳෝ", "අපොයි", "අපෝ", "අයියෝ", "ආයි", "ඌයි", "චී", "චිහ්", "චික්", "හෝ‍", "දෝ", "දෝහෝ", "මෙන්", "සේ", "වැනි", "බඳු", "වන්", "අයුරු", "අයුරින්", "ලෙස", "වැඩි", "ශ්‍රී", "හා", "ය", "නිසා", "නිසාවෙන්", "බවට", "බව", "බවෙන්", "නම්", "වැඩි", "සිට", "දී", "මහා", "මහ", "පමණ", "පමණින්", "පමන", "වන", "විට", "විටින්", "මේ", "මෙලෙස", "මෙයින්", "ඇති", "ලෙස", "සිදු", "වශයෙන්", "යන", "සඳහා", "මගින්", "හෝ‍", "ඉතා", "ඒ", "එම", "ද", "අතර", "විසින්", "සමග", "පිළිබඳව", "පිළිබඳ", "තුළ", "බව", "වැනි", "මහ", "මෙම", "මෙහි", "මේ", "වෙත", "වෙතින්", "වෙතට", "වෙනුවෙන්", "වෙනුවට", "වෙන", "ගැන", "නෑ", "අනුව", "නව", "පිළිබඳ", "විශේෂ", "දැනට", "එහෙන්", "මෙහෙන්", "එහේ", "මෙහේ", "ම", "තවත්", "තව ", "සහ",
                     "දක්වා", "ට", "ගේ", "එ", "ක", "ක්", "බවත්", "බවද", "මත", "ඇතුලු", "ඇතුළු", "මෙසේ", "වඩා", "වඩාත්ම", "නිති", "නිතිත්", "නිතොර", "නිතර", "ඉක්බිති", "දැන්", "යලි", "පුන", "ඉතින්", "සිට", "සිටන්", "පටන්", "තෙක්", "දක්වා", "සා", "තාක්", "තුවක්", "පවා", "ද", "හෝ‍", "වත්", "විනා", "හැර", "මිස", "මුත්", "කිම", "කිම්", "ඇයි", "මන්ද", "හෙවත්", "නොහොත්", "පතා", "පාසා", "ගානෙ", "තව", "ඉතා", "බොහෝ", "වහා", "සෙද", "සැනින්", "හනික", "එම්බා", "එම්බල", "බොල", "නම්", "වනාහි", "කලී", "ඉඳුරා", "අන්න", "ඔන්න", "මෙන්න", "උදෙසා", "පිණිස", "සඳහා", "අරබයා", "නිසා", "එනිසා", "එබැවින්", "බැවින්", "හෙයින්", "සේක්", "සේක", "ගැන", "අනුව", "පරිදි", "විට", "තෙක්", "මෙතෙක්", "මේතාක්", "තුරු", "තුරා", "තුරාවට", "තුලින්", "නමුත්", "එනමුත්", "වස්", "මෙන්", "ලෙස", "පරිදි", "එහෙත්"]


##### Load the dataset


In [ ]:

df = pd.read_csv("./PublicFigureStatementsSinglish.xls", encoding="utf-16")
df.head(10)


##### Filtering dataset into positives and negatives


In [ ]:
all_positive_tweets = df[df["Impact"] ==
                         "Positive"]["Statement"].values.tolist()
all_negative_tweets = df[df["Impact"] ==
                         "Negative"]["Statement"].values.tolist()

print("Number of positive tweets: ", len(all_positive_tweets))
print("Number of negative tweets: ", len(all_negative_tweets))


##### Randomly choosing 300 samples for each positives and negatives


In [ ]:
dataset_size = 300
all_positive_tweets = np.random.choice(
    all_positive_tweets, size=dataset_size, replace=False).tolist()
all_negative_tweets = np.random.choice(
    all_negative_tweets, size=dataset_size, replace=False).tolist()


print("Number of positive tweets: ", len(all_positive_tweets))
print("Number of negative tweets: ", len(all_negative_tweets))


##### Tweet processing function depending on the language to translate to


In [ ]:
def process_tweet(tweet, language="si"):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
        language: language to translate to "si" or "en"
    Output:
        tweets_clean: a list of words containing the processed tweet
    """

    english_stemmer = PorterStemmer()
    sinhala_stemmer = SinhalaStemmer()

    stopwords_english = stopwords.words("english")

    # remove stock market tickers like $GE
    tweet = re.sub(r"\$\w*", "", tweet)

    # remove old style retweet text "RT"
    tweet = re.sub(r"^RT[\s]+", "", tweet)

    # remove hyperlinks
    tweet = re.sub(r"(http|https|ftp):\/\/(\S*)", "", tweet)

    # remove hashtag sign from words
    tweet = tweet.replace("#", "")

    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    translator = Translator()

    # remove stopwords and punctuation
    def filter_english(
        word): return word not in stopwords_english and word not in string.punctuation

    def filter_sinhala(
        word): return word not in stopwords_sinhala and word not in string.punctuation

    def stem_english(word):
        if (filter_english(word)):
            stem_word = english_stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    def stem_sinhala(word):
        if (filter_sinhala(word)):
            stem_word = sinhala_stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word[0])

    if language == "en":

        for word in tweet_tokens:
            detected_language = translator.detect(word)
            if "en" in detected_language.lang:  # parse English word
                stem_english(word)
            elif "si" in detected_language.lang:  # parse Sinhala word
                if filter_sinhala(word):
                    translated_text = translator.translate(word)
                    tokenized_text = tokenizer.tokenize(translated_text.text)
                    for word in tokenized_text:
                        stem_english(word)

    elif language == "si":

        for word in tweet_tokens:
            detected_language = translator.detect(word)
            if "en" in detected_language.lang:  # parse English word
                if (filter_english(word)):
                    translated_text = translator.translate(word)
                    tokenized_text = tokenizer.tokenize(translated_text.text)
                    for word in tokenized_text:
                        stem_sinhala(word)
            elif "si" in detected_language.lang:  # parse Sinhala word
                stem_sinhala(word)

    return tweets_clean


##### Word frequency dictionary `(word, label): frequency`


In [ ]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs


In [ ]:
tweets = all_positive_tweets + all_negative_tweets
labels = np.append(np.ones((len(all_positive_tweets))),
                   np.zeros((len(all_negative_tweets))))

freqs = build_freqs(tweets, labels)


In [ ]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# avoid assumptions about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))


In [ ]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0

    n = freqs.get((word, label), 0)

    return n


In [ ]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation.
    '''
    loglikelihood = {}
    logprior = 0

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = V_pos = V_neg = 0

    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:
            # increment the count of unique positive words by 1
            V_pos += 1

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:
            # increment the count of unique negative words by 1
            V_neg += 1

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    # Calculate D_neg, the number of negative documents
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)

    return logprior, loglikelihood


In [ ]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)

print(logprior)
print(len(loglikelihood))


In [ ]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p


In [ ]:
my_tweet = 'බබා කෝලි නම් ගේමක් නෑ'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)


In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    sum = 0
    for i, y in enumerate(test_y):
        sum += np.abs(y - y_hats[i])
    error = sum / len(test_y)

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    return accuracy


In [ ]:
print("Naive Bayes accuracy = %0.4f" % (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))